In [2]:
import cv2
import numpy as np

# Wczytanie pliku wideo
cap = cv2.VideoCapture('OKNO_4.avi')

# Zmienne pomocnicze

shape_detected = False
previous_x, previous_y = 0,0
licznik = 0
dobre_8 = 0
zle_8 = 0
dobre_u = 0
zle_u = 0
area = 0
rodzaj = ''

# Odpowiednie granice dla wykrywanych kolorow
lower_pink = np.array([150,50,50])
upper_pink = np.array([180,255,255])

lower_green = np.array([240, 80, 80])
upper_green = np.array([255, 255, 255])

lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

# Ustalenie przedzialow pol powierzchni wykrywanych elementow
marg = 1.0
upper_8 = 1099.5 
lower_8 = 1086.5
upper_8g = 830.0 + marg
lower_8g = 820.0 - marg
upper_8d = 1086.5
lower_8d = 1083.0

upper_u = 630.0 + marg
lower_u = 590.0 - marg
upper_ul = 1120.0 + marg
lower_ul = 1100.0 - marg
upper_up = 1065.0 + marg
lower_up = 1020.0 - marg    

while True:
    
    ret, frame = cap.read()
    
    # Sprawdzenie czy plik sie nie skonczyl lub nie doszlo do innego bledu
    if not ret:
        break
    
    # zmiana przestrzeni kolorow z BGR na GRAY
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Pobranie wymiarów okna 
    height, width = gray.shape[:2]

    # Obliczanie wymiarów ROI
    roi_width = int(width)
    roi_height = int(height / 2)

    # Obliczanie położenia w oknie
    roi_x = int((width - roi_width) / 2)
    roi_y = int((height - roi_height)/2)
    
    # Pobranie fragmentu obrazu z zadeklarowanego ROI
    roi = cv2.getRectSubPix(gray, (roi_width, roi_height), (roi_x + roi_width / 2, roi_y + roi_height / 1000))

    # Wykrywanie krawędzi przy pomocy filtra Canny
    edges = cv2.Canny(roi, 50, 50)

    # Znajdowanie konturów
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Iterowanie po konturach
    for cnt in contours:

        # Obliczanie właściwości dla kazdego wykrytego konturu
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)

        if area < 3000 and area > 400:
            # Funkcja zwracajaca polozenie lewego gornego naroznika oraz szerokosc i wysokosc konturu
            (x, y, w, h) = cv2.boundingRect(cnt)
            
            # Sprawdzenie czy kolejny wykryty ksztalt znajduje sie w innym miejscu na ekranie w celu unikniecia wielokrotnego wykrywania tego samego ksztaltu
            if abs(previous_x - x) == 0 and abs(previous_y - y) != 0:
                shape_detected = False  #Flaga pomocnicza
                continue

            # Nadpisanie przetwarzanych wspolrzednych pomocniczych
            previous_x, previous_y = x, y

            # Rysowanie prostokatnego obrysu dookola ksztaltu
            obrys = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), 0)          #kolor zielony  (0, 255, 0)
        
            # Ustawienie stałych wymiarów okna
            width, height = 200, 260

            # Wyświetlenie wykrytego kształtu w powyzej zadeklarowanym oknie
            shape_img = cv2.resize(frame[y:y+h, x:x+w], (width, height))

            # Zmiana przestrzeni barw z BGR na HSV
            hsv = cv2.cvtColor(shape_img, cv2.COLOR_BGR2HLS)
            #Przedzialy dla kazdego rodzaju koloru
            pink_mask = cv2.inRange(hsv, lower_pink, upper_pink)
            green_mask = cv2.inRange(hsv, lower_green, upper_green)
            blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
            # Zliczanie niezerowych (nieczarnych) pikseli w masce
            pink_area = cv2.countNonZero(pink_mask)
            green_area = cv2.countNonZero(green_mask)
            blue_area = cv2.countNonZero(blue_mask)

            # Porownywanie powyzszych wynikow w celu ustalenia koloru ksztaltu
            color = "rozowy" if pink_area > green_area and pink_area > blue_area else "seledynowy" if green_area == blue_area and blue_area == pink_area else "niebieski"

            # Warunkowe sprawdzanie jaki ksztalt jest wyswietlany
            if (area >= lower_8 and area <= upper_8) or (area == 1084.5 and (perimeter < 149.0 and perimeter > 148.0)) or (area == 1086.0 and color == 'niebieski'):
                rodzaj = 'osiem'
            elif (area >= lower_8g and area <= upper_8g) or (area >= lower_8d and area <= upper_8d) or (area == 1086.5 and color == 'niebieski') or (area == 1086.0 and color == 'różowy'):
                rodzaj = 'wadliwe osiem' 
            elif area >= lower_u and area <= upper_u:
                rodzaj = 'litera u'
            elif (area >= lower_ul and area <= upper_ul) or (area >= lower_up and area <= upper_up):
                rodzaj = 'wadliwa litera u'
            
            # Dodatkowe wyswietlanie wykrywanych krawedzi ksztaltu
            edges2 = cv2.Canny(shape_img, 50, 150)
            cv2.imshow("krawedzie", edges2)

            # Tworzenie nowego okna do wyswietlania stanu aktualnego
            cv2.namedWindow("Stan aktualny", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("Stan aktualny", 700, 50)

            # Tworzenie czarnego obrazu o rozmiarach 700x50 
            stan = np.zeros((50, 700, 3), dtype=np.uint8)

            # zliczanie wykrytych kształtów i printowanie informacji o nich do konsoli
            if not shape_detected:                                                                                                      #         DORZUCIC ZLICZANIE KOLOROW
                licznik += 1
                # Printowanie aktualnych wynikow przetwarzania wideo
                print('Rozpoznany znak nr ' + str(licznik) + " w kolorze " + color + 'm. Jest to ' + rodzaj +'.')
                # Zliczanie ksztaltow
                if rodzaj == 'osiem':
                    dobre_8 += 1
                elif rodzaj == 'wadliwe osiem':
                    zle_8 += 1
                elif rodzaj == 'litera u':
                    dobre_u += 1
                elif rodzaj == 'wadliwa litera u':
                    zle_u += 1
                # Flaga pomocnicza
                shape_detected = True
            
            # Wyswietlanie ksztaltu w nowym oknie o dopasowanych do niego rozmiarach
            cv2.imshow("Detected Shape", shape_img)

            # Wyswietlanie tekstu o aktualnie rozpoznanym znaku w nowym oknie
            cv2.putText(stan, 'Rozpoznany znak nr ' + str(licznik) + " w kolorze " + color + 'm. Jest to ' + rodzaj +'.', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.imshow("Stan aktualny", stan)

    # Wyswietlanie glowengo pliku wideo, Obszaru ROI oraz wykrywnaych w nim krawedzi ksztaltow
    cv2.imshow('frame', frame)
    cv2.imshow('edges', edges)
    cv2.imshow('roi', roi)

    # Realizowanie pauzy przez nacisniecie spacji oraz wyjscia z programu przez nacisniecie klawisza 'q'
    if cv2.waitKey(1) & 0xFF == ord(' '):       # Jeśli klawisz spacji zostanie naciśnięty nastepuje wstrzymanie filmu
            while True:
                key2 = cv2.waitKey(1)
                if key2 == ord(' '):            # Jeśli klawisz spacji zostanie naciśnięty ponownie nastepuje wznowienie filmu
                    break                       

    elif cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

# Printowanie raportu koncowego o wykrywanych elementach
print('Razem rozpoznano ' + str(licznik) + ' znaków. \n' + 'Wśród nich było: \n' + str(dobre_8) + ' prawidłowych ósemek,\n' + str(zle_8) + ' wadliwych ósemek,\n' + str(dobre_u) + ' prawidłowych liter U,\n' + str(zle_u) + ' wadliwych liter U.')

# Tworzenie nowego okna do wyswietlania raportu
cv2.namedWindow("Raport", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Raport", 300, 200)

# Tworzenie czarnego obrazu o rozmiarach 300x200 
end_img = np.zeros((200, 300, 3), dtype=np.uint8)

# Dodanie tekstu  do obrazu
cv2.putText(end_img, 'Razem rozpoznano ' + str(licznik) + ' znakow.', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(end_img, 'Wsrod nich bylo: ', (30, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(end_img, str(dobre_8) + ' prawidlowych osemek,', (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(end_img, str(zle_8) + ' wadliwych osemek,', (30, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(end_img, str(dobre_u) + ' prawidlowych liter U,', (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(end_img, str(zle_u) + ' wadliwych liter U.', (30, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

# Zamkniecie okien programu
cap.release()
cv2.destroyAllWindows()

# Wyświetlenie obrazu w oknie "Koniec"
cv2.imshow("Raport", end_img)

# Waronki zamkniecia okra z raportem
if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(0):
    cv2.destroyAllWindows()

    

Rozpoznany znak nr 1 w kolorze niebieskim. Jest to litera u.
Rozpoznany znak nr 2 w kolorze rozowym. Jest to wadliwa litera u.
Rozpoznany znak nr 3 w kolorze niebieskim. Jest to osiem.
Rozpoznany znak nr 4 w kolorze seledynowym. Jest to osiem.
Rozpoznany znak nr 5 w kolorze seledynowym. Jest to litera u.
Rozpoznany znak nr 6 w kolorze niebieskim. Jest to wadliwa litera u.
Rozpoznany znak nr 7 w kolorze rozowym. Jest to wadliwa litera u.
Rozpoznany znak nr 8 w kolorze seledynowym. Jest to osiem.
Rozpoznany znak nr 9 w kolorze seledynowym. Jest to osiem.
Rozpoznany znak nr 10 w kolorze seledynowym. Jest to wadliwe osiem.
Rozpoznany znak nr 11 w kolorze niebieskim. Jest to litera u.
Rozpoznany znak nr 12 w kolorze seledynowym. Jest to osiem.
Rozpoznany znak nr 13 w kolorze seledynowym. Jest to litera u.
Rozpoznany znak nr 14 w kolorze rozowym. Jest to wadliwa litera u.
Rozpoznany znak nr 15 w kolorze rozowym. Jest to wadliwe osiem.
Rozpoznany znak nr 16 w kolorze niebieskim. Jest to osiem